In [ ]:
%matplotlib inline

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import random

simulator = AerSimulator()

def make_branch_circuit(branch_type: str) -> QuantumCircuit:
    """
    branch_type: 'Z' or 'X' — 기능 분기 형태를 수동 지정
    """
    qr = QuantumRegister(2)
    cr = ClassicalRegister(2)
    qc = QuantumCircuit(qr, cr)

    qc.h(0)
    qc.cx(0, 1)

    if branch_type == 'Z':
        qc.z(1)
    elif branch_type == 'X':
        qc.x(1)

    qc.measure([0, 1], [0, 1])
    return qc

# 실험 회로: 관측 후 조건에 따라 분기하는 대신, 분기를 나눠 실행
circuit_exp_z = make_branch_circuit('Z')  # 구조 선택이 Z였다고 가정
circuit_exp_x = make_branch_circuit('X')  # 구조 선택이 X였다고 가정

# 대조 회로: 구조 없이 랜덤 선택
circuit_ctrl_z = make_branch_circuit('Z')
circuit_ctrl_x = make_branch_circuit('X')

# 실행
job_exp_z = simulator.run(circuit_exp_z, shots=1024)
job_exp_x = simulator.run(circuit_exp_x, shots=1024)
job_ctrl_z = simulator.run(circuit_ctrl_z, shots=1024)
job_ctrl_x = simulator.run(circuit_ctrl_x, shots=1024)

# 결과
result_exp_z = job_exp_z.result().get_counts()
result_exp_x = job_exp_x.result().get_counts()
result_ctrl_z = job_ctrl_z.result().get_counts()
result_ctrl_x = job_ctrl_x.result().get_counts()

# 시각화
plot_histogram(
    [result_exp_z, result_exp_x],
    legend=['Experimental-Z', 'Experimental-X']
)
plt.title("G-F Experimental: Functional Branches")
plt.show()

plot_histogram(
    [result_ctrl_z, result_ctrl_x],
    legend=['Control-Z', 'Control-X']
)
plt.title("G-F Control: Random Functional Branches")
plt.show()
